In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os
import seaborn
from tqdm import tqdm

trainPath = 'data/Dataset_1_train_asset.csv'
targetPath = 'data/Dataset_1_train_payoff.csv'
testPath = 'data/Dataset_1_test_asset.csv'
growthPath = 'data/Dataset_1_train_asset_daily growth_rate.csv'

trainData = pd.read_csv(trainPath, header=None)
testData = pd.read_csv(testPath, header=None)
target = pd.read_csv(targetPath, header=None)
growthTrain = pd.read_csv(growthPath)

In [ ]:
# exercise 1 - BlackScholes Model

N = norm.cdf

def bs_call(K: float, #underlying asset's price
            T: float, #time till maturity
            S: float = 1.0, # strike price
            r: float = 0.0,  # risk-free
            sigma: float = 0.158):
    
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2), N(-d1)

def bs_put(K: float, #underlying asset's price
           T: float, #time till maturity
           S: float = 1.0, #strike price
           r: float = 0.0, # risk-free
           sigma: float = 0.158):
    
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

In [ ]:
def calculate_hedge(df: pd.DataFrame):
    deltas = pd.DataFrame(index=range(df.shape[0]),columns=range(df.shape[1]))
    
    for i in tqdm(range(0, df.shape[0])):
        short_position = 0.0
        for j in range(0, df.shape[1]):
            S = 100
            initial_price = 100
            K = df.loc[i, :].values[j]
            T = (31 - j)/365
            result, delta = bs_call(K=K, T=T, S=S)
            short_position += -result
            deltas.loc[i, j] = delta

    return deltas


In [ ]:
def col_numeric_names(df):
    df = df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
    return df

def absolute_growth(df):
    len_df = df.shape[1]
    temp = df.iloc[:,1:len_df]
    temp = col_numeric_names(temp)
    temp2 = df.iloc[:,:len_df-1]
    df2 = temp-temp2
    df2.insert(0, "0", 0)
    df2 = col_numeric_names(df2)
    return df2
growthTrain = absolute_growth(trainData)

In [ ]:
bs_deltas = calculate_hedge(trainData)
bs_deltas_test = calculate_hedge(testData)

In [ ]:
# multi in and output RNN attempt:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(42)
# Set up device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define stock price path, payoffs, and transaction costs
S_list = torch.Tensor(trainData.values.tolist())
S_test = torch.Tensor(testData.values.tolist())
payoffs = torch.Tensor([item for sublist in target.values for item in sublist])
transaction_costs = torch.Tensor([0.0] * S_list.shape[0])
transaction_costs_test = torch.Tensor([0.0] * S_test.shape[0])
growths = torch.Tensor(growthTrain.values.tolist())

# Set hyperparameters
batch_size = 256
num_epochs = 50
learning_rate = 0.001

# Set up dataset and iterator
dataset_train = torch.utils.data.TensorDataset(S_list[0:7999], payoffs[0:7999], transaction_costs[0:7999], growths[0:7999])
dataset_val = torch.utils.data.TensorDataset(S_list[7999:], payoffs[7999:], transaction_costs[7999:], growths[7999:])
dataset_test = torch.utils.data.TensorDataset(S_test, transaction_costs)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False)

class DeltaRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DeltaRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, S):
        #S = S.unsqueeze(1)
        rnn_out, _ = self.rnn(S)
        delta_nn_output = self.fc(rnn_out)
        return delta_nn_output.squeeze()

In [ ]:
class BS_PnLLoss(torch.nn.Module):
    def __init__(self, beta=1):
        super(BS_PnLLoss, self).__init__()
        self.beta = beta
        self.q1 = torch.tensor((0.), dtype=torch.float32, requires_grad=True, device=device)
        self.q2 = torch.tensor((0.), dtype=torch.float32, requires_grad=True, device=device)

    def forward(self, predicted, true, growth):
        # calculate delta_nn_output
        S_delta_T = torch.transpose(growth, 0, 1)
        zeros = torch.zeros((predicted.shape[0], 1)).to(device)
        delta_nn_output = torch.cat([zeros, predicted.squeeze(-1)], dim=1)
        delta_S = torch.diagonal(torch.matmul(delta_nn_output, S_delta_T))
        # calculate CT
        c = 0 # for now with black scholes
        CT = c * torch.sum(torch.abs(torch.diff(delta_nn_output, dim=1)))
        # calculate STK
        STK = true
        # calculate PnL
        PnL = CT + delta_S - STK
        L = -PnL
        # calculate risk measure
        #ES50 = torch.median(L)
        #ES99 = np.percentile(L.cpu().detach().numpy(), q=1)
        #risk_measure = 1/(1+self.beta) * (ES50 + self.beta * ES99)
        es_99 = (F.relu(L-self.q1).mean())/(1-0.99) + self.q1
        es_50 = (F.relu(L-self.q2).mean())/(1-0.5) + self.q2
        risk_measure = (es_50 + es_99*self.beta)/(1+self.beta)
        return risk_measure

In [ ]:
import torch.nn.functional as F
import math

train_loss = []
test_loss = []
model = DeltaRNN(1, 16, 1).to(device)
# Set up optimizer

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = BS_PnLLoss(beta=1)

for epoch in tqdm(range(num_epochs), desc='Training'):
    model.train()
    running_loss = 0.0
    counter = 0
    for S, y, c, g in train_loader:
        counter += 1
        S = S.to(device).unsqueeze(-1)
        S_in = S[:, :-1].to(device)
        y = y.to(device)
        c = c.to(device)
        g = g.to(device)

        # Calculate gradients and update model weights
        optimizer.zero_grad()
        deltas = model(S_in)
        losses = criterion(deltas, y, g)
        running_loss += losses.item()

        losses.backward()
        optimizer.step()

    epoch_train_loss = running_loss / counter
    train_loss.append(epoch_train_loss)
    print("Train loss: {}".format(epoch_train_loss))
    
    running_loss_test = 0.0
    counter = 0

    for S_val, y_val, c_val, g_val in val_loader:
        model.eval()
        
        counter += 1
        S_val = S_val.to(device).unsqueeze(-1)
        S_in_val = S_val[:, :-1].to(device)
        y_val = y_val.to(device)
        c_val = c_val.to(device)
        g_val = g_val.to(device)

        deltas_val = model(S_in_val)
        losses_test = criterion(deltas_val, y_val, g_val)
        running_loss_test += losses_test.item()

    epoch_test_loss = running_loss_test / counter
    test_loss.append(epoch_test_loss)
    print("Test loss: {}".format(epoch_test_loss))

In [ ]:
final = []
for stock, costs in tqdm(test_loader):
    stock = stock.to(device).unsqueeze(-1)
    costs = costs.to(device)
    final.append(model(stock).cpu().detach().numpy())

In [ ]:
def StandardData(data: np.ndarray):
    min_value = np.min(data)
    max_value = np.max(data)
    normalized_data = (data - min_value) / (max_value - min_value)
    return normalized_data

In [ ]:
def plot_figures(deltas_BS: pd.DataFrame, rnn_bs: pd.DataFrame, df: pd.DataFrame): 
    fig, axs = plt.subplots(2, 3)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Delta spread at different time intervals', size=45)

    axs[0,0].plot(df.loc[:, 0], deltas_BS.loc[:, 0], 'bo', label='Black-Scholes')
    axs[0,0].plot(df.loc[:, 0], rnn_bs.loc[:, 0], 'gx', label='Delta-RNN')
    axs[0,0].legend()
    axs[0,0].set_title('Time 0')
    axs[0,0].set_ylim([0, 1])
    axs[0,1].plot(df.loc[:, 1], deltas_BS.loc[:, 1], 'bo', label='Black-Scholes')
    axs[0,1].plot(df.loc[:, 1], StandardData(rnn_bs.loc[:, 1]), 'gx', label='Delta-RNN')
    axs[0,1].legend()
    axs[0,1].set_title('Time 1')
    axs[0,2].plot(df.loc[:, 5], deltas_BS.loc[:, 5], 'bo', label='Black-Scholes')
    axs[0,2].plot(df.loc[:, 5], StandardData(rnn_bs.loc[:, 5]), 'gx', label='Delta-RNN')
    axs[0,2].legend()
    axs[0,2].set_title('Time 5')
    axs[1,0].plot(df.loc[:, 15], deltas_BS.loc[:, 15], 'bo', label='Black-Scholes')
    axs[1,0].plot(df.loc[:, 15], StandardData(rnn_bs.loc[:, 15]), 'gx', label='Delta-RNN')
    axs[1,0].legend()
    axs[1,0].set_title('Time 15')
    axs[1,1].plot(df.loc[:, 25], deltas_BS.loc[:, 25], 'bo', label='Black-Scholes')
    axs[1,1].plot(df.loc[:, 25], StandardData(rnn_bs.loc[:, 25]), 'gx', label='Delta-RNN')
    axs[1,1].legend()
    axs[1,1].set_title('Time 25')
    axs[1,2].plot(df.loc[:, 30], deltas_BS.loc[:, 30], 'bo', label='Black-Scholes')
    axs[1,2].plot(df.loc[:, 30], StandardData(rnn_bs.loc[:, 30]), 'gx', label='Delta-RNN')
    axs[1,2].legend()
    axs[1,2].set_title('Time 30')

    for ax in axs.flat:
        ax.set(xlabel='Spot price', ylabel='Delta')

In [ ]:
plot_figures(bs_deltas_test, pd.DataFrame(final), testData)